In [ ]:
using DataFrames
using Statistics
using CSV

In [ ]:
function inicializar()
    out = DataFrame(Hora = String[], kWhcount = Float64[], count = Int64[])
    rename!(out, [:"Hora", :"kWh*count", :"count"])

    for i in 0:23
        for j in 0:3
            append!(out[!, "Hora"],[hora(i, j)])
            append!(out[!, "kWh*count"],[0.0])
            append!(out[!, "count"],[0])
        end
    end    
    
    return out
end

In [ ]:
function hora(i, j)      
    if i != 0
        if i < 10
            h = "0" * string(i)
        else
            h = string(i)
        end
    else
        h = "00"  
    end   

    if j != 0
        m = string(j*15)
    else
        m = "00"    
    end   
    
    return h * ":" * m * ":" * "00"
end

In [ ]:
function sumar_columnas(out, df)
    
    if !(isempty(out)) || !(isempty(df))
            out[!, 2] .= out[!, 2] + df[!, 2]
            out[!, 3] .= out[!, 3] + df[!, 3]
        return out
    end
end

In [ ]:
function df_por_tarifa(df)
    aux = DataFrame()
    out = DataFrame(Hora = String[], kWhcount = Float64[], count = Int64[])
    rename!(out, [:"Hora", :"kWh*count", :"count"])

    for i in 0:23
        for j in 0:3
            horario = hora(i, j)
            aux = filter(:"Hora" => n -> n == horario, df)

            kWh = mean(aux[!, 2])
            count = length(aux[!, 2])
            prod = kWh*count

            #push!(aux, [horario prod count])
            append!(out[!, "Hora"],[horario])
            append!(out[!, "kWh*count"],[prod])
            append!(out[!, "count"],[count])        
        end
    end
    return out
end

In [ ]:
# obtener tarifa
function t_n_f_index(archivo)
    open(archivo, "r") do f
        lines = readlines(f)

        tariff_index = findfirst(t -> occursin("Servicio:", t), lines) # indices de tarifa y fecha
                             
        # obtener tipo de tarifa
        tariff_vector = split(lines[tariff_index], ";")
        filter!(x -> x != " ", tariff_vector) # eliminar espacio vacio 
        filter!(x -> x != "", tariff_vector) # eliminar espacio vacio 
        
        # tarifa e indice de fecha
        return tariff_vector[2]
    end
end

In [ ]:
function limpiarTexto(f)

    lines = readlines(f)
    fecha_in = findfirst(t -> occursin("Fecha", t), lines)
    lines[end] = "" # eliminar ultima linea
    # si existe esa esctructura eliminarla
    if lines[end - 1] == ";;;;;;;;;;;;;;;;"
        lines[end - 1] = "" 
    end

    lines = lines[fecha_in: end] # obtener solo las lineas a partir de la fecha
    lines[1] = replace(lines[1], "\xe1" => "") # sustituir caracteres especiales  
    string = join(lines, "\n") # vector a string para leerlo como csv
    string = replace(string, ";;" => ";") # eliminar duplicados
    
    return string
end

In [ ]:
function primeraLimpiezaDF(df)
    # renombrar "Fecha hora" a "Fecha Hora"
    if "Fecha hora" in names(df)
        rename!(df, "Fecha hora" => "Fecha Hora")
    end
    select!(df, "Fecha Hora", "kWh") # eliminar las columnas no utiles
    dropmissing!(df, "Fecha Hora") # eliminar los missing
    dropmissing!(df, "kWh") # eliminar los missing
    
    return df
end

In [ ]:
function segundaLimpiezaDF(df)
    df[!,2] .= replace.(df[!,2], "," => ".") # coma a punto
    df[!,2] .= parse.(Float64, df[!,2]) # string a float

    transform!(df, :"Fecha Hora" => ByRow(x -> split(x, ' ')) => [:"Fecha", :"Hora"]) # separar fecha y hora
    df = df[:, [2, 4]] # dejar las columnas Fecha y Fecha Hora
    df[!, 1], df[!, 2] = df[!, 2], df[!, 1] # cambiar valores de las columnas
    rename!(df, [:"Hora", :"kWh"]) # renombrar columnas
    
    return df
end

In [ ]:
# obtener ubicacion de todos los archivos
archivos = Vector{String}()
carpeta = "procesar"
# recorrer cada carpeta dentro de "carpeta"
for folder in readdir(carpeta)
    
    # recorrer cada archivo dentro de la carpeta dentro de "folder"
    for file in readdir(carpeta * "/" *folder)
        push!(archivos, "$carpeta/$folder/$file")
    end
    
end

In [ ]:
# obtener vectores de tarifa e indice de fecha
tariff = Vector{String}()

for i in 1:length(archivos)

    archivo = archivos[i]
    aux = t_n_f_index(archivo)
    # guardar tarifa e indice de fecha
    push!(tariff, aux)

end

In [ ]:
tariff_set = Set(tariff)
tariff_set_list = collect(tariff_set)
#B = [(i, count(==(i), tarif)) for i in unique(tarif)]
for i in unique(tariff)
    println("$i -> ", count(==(i), tariff))
end

In [ ]:
# para cada tarifa
contador = 0
# recorrer elementos unicos de tarifa
for i in 1:length(tariff_set_list)

    # se opera sobre cada coincidencia de tarifa
    # sin procesar "Medicion:"
    if occursin("Medicion:", tariff_set_list[i]) != true
        global out = inicializar() # inicializar un dataframe
        posicion_tarifas = findall(x -> x == tariff_set_list[i] , tariff) # posiciones de tarifa    

        # recorrer cada archivo de cada tarifa
        for indice_tarifa in posicion_tarifas
            archivo = archivos[indice_tarifa]
            
                open(archivo, "r") do f

                    string = limpiarTexto(f)
                    df = CSV.read(IOBuffer(string), DataFrame; delim = ";", header = true) # string a csv                    
                    df = primeraLimpiezaDF(df)

                    if length(df[!, 1]) > 100
                        
                        df = segundaLimpiezaDF(df)
                        df_out = df_por_tarifa(df)
                        for col in eachcol(df_out)
                            replace!(col, NaN => 0.0)
                        end    
                        out = sumar_columnas(df_out, out)
                    
                    end 
                end
        end
        nombre = "$(tariff_set_list[i])-$(count(==(tariff_set_list[i]), tariff))"
        CSV.write("excel_out/$nombre.csv", out)
    end
end
""

2200 archivos -> 17m 43s

2000 archivos -> 15m 27.9s

1500 archivos -> 13m 26.6s

1000 archivos -> 7m 55.1s

500 archivos -> 5m 13s

---------------------------------------------------------------------------------------------------------------------------------------------------